In [ ]:
import $ivy.`org.typelevel::cats-core:2.1.0`

// These are all the imports you need for everything here
import cats.implicits._
import cats.Eval

# The Eval Monad
`cats.Eval` is a monad that allows us to abstract over different *models of evaluation*. It abstracts over 3 models *eager*, *lazy*, and *memoized*.

In [ ]:
// Eager and memoized
val x = {
  println("Computing X")
  math.random
}

In [ ]:
x

In [ ]:
// Lazy and not memoized
def y = {
  println("Computing Y")
  math.random
}

In [ ]:
y

In [ ]:
lazy val z = {
  println("Computing Z")
  math.random
}

In [ ]:
z

`Eval` has three subtypes: `Now`, `Later`, and `Always`.

In [ ]:
val now = Eval.now {
  println("Computing now")
  math.random
}

val later = Eval.later {
  println("Computing later")
  math.random
}

val always = Eval.always {
  println("Computing always")
  math.random
}

In [ ]:
val nowValue = now.value

In [ ]:
val laterValue = later.value

In [ ]:
val alwaysValue = always.value

Like all monads, `Eval`'s `map` and `flatMap` methods add computations to a chain. In this case, however, the chain is stored explicitly as a list of functions. The functions aren’t run until we call `Eval`'s `value` method to request a result:

In [ ]:
val greeting = Eval.
  now { println("Step 1"); "Hello" }.
  map { str => println("Step 2"); s"$str world" }

In [ ]:
greeting.value

Note that, while the semantics of the originating `Eval` instances are maintained, mapping and flatMapping functions are always caled lazily and on demand (`def` semantics).

`Eval` has a `memoize` method that allows us to memoize a chain of computations. The result of the chain up to the call to `memoize` is cached, whereas calculations after the call retain their original semantics:

In [ ]:
val saying = Eval.
  always { println("Step 1"); "The cat" }.
  map { str => println("Step 2"); s"$str sat on" }.
  memoize.
  map { str => println("Step 3"); s"$str the mat" }

In [ ]:
saying.value

A useful property of `Eval` is that it's `map` and `flatMap` methods are *trampolined*. This means that we can nest calls to `map` and `flaMap` without consuming stack frames or blowing the stack. `Eval` is stack safe.

In [ ]:
def factorial(n: BigInt): BigInt =
  if(n == 1) n else n * factorial(n - 1)

factorial(50000)

In [ ]:
// Let's try that again
def factorial(n: BigInt): Eval[BigInt] =
  if(n == 1) {
    Eval.now(n)
  } else {
    factorial(n - 1).map(_ * n)
  }

factorial(50000).value

In [ ]:
def factorial(n: BigInt): Eval[BigInt] =
  if(n == 1) {
    Eval.now(n)
  } else {
    Eval.defer(factorial(n - 1).map(_ * n))
  }

factorial(50000).value

`Eval.defer` takes an existing instance of `Eval` and defers its evaluation. The `defer` method is trampolined like `map` and `flatMap`, so we can use it as a quick way to make an existing operation stack safe. We must remember that trampolining is not free. It avoids consuming stack by creating a chain of function objects on the head. There is still a limit, it's just based on the size of the heap rather than the stack.

## Exercise
Let's make `foldRight` stack safe using `Eval`:

In [ ]:
def foldRight[A, B](as: List[A], acc: B)(fn: (A, B) => B): B =
  as match {
    case head :: tail =>
      fn(head, foldRight(tail, acc)(fn))
    case Nil =>
      acc
  }

val list = List.fill(50000)(1)

foldRight(list, 0)(_ + _)

In [ ]:
def foldRight[A, B](as: List[A], acc: B)(fn: (A, B) => B): B =
  foldRightEval(as, Eval.now(acc)) { (a, b) =>
    b.map(fn(a, _))
  }.value

def foldRightEval[A, B](as: List[A], acc: Eval[B])(fn: (A, Eval[B]) => Eval[B]): Eval[B] =
  as match {
    case head :: tail =>
      Eval.defer(fn(head, foldRightEval(tail, acc)(fn)))
    case Nil =>
      acc
  }

val list = List.fill(50000)(1)

foldRight(list, 0)(_ + _)